In [41]:
import os, math, subprocess
import datetime as dt

#data wrangling
import numpy as np
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#excel library
import xlrd

In [53]:
#Loading data
df_newcustomerlist = pd.read_excel('KPMG.xlsx', sheet_name= 'NewCustomerList')
df_newcustomerlist.head(5)

first_name  last_name  gender  past_3_years_bike_related_purchases  \
0    Chickie    Brister    Male                                   86   
1      Morly     Genery    Male                                   69   
2    Ardelis  Forrester  Female                                   10   
3     Lucine      Stutt  Female                                   64   
4    Melinda     Hadlee  Female                                   34   

         DOB                   job_title job_industry_category  \
0 1957-07-12             General Manager         Manufacturing   
1 1970-03-22         Structural Engineer              Property   
2 1974-08-28      Senior Cost Accountant    Financial Services   
3 1979-01-28  Account Representative III         Manufacturing   
4 1965-09-21           Financial Analyst    Financial Services   

      wealth_segment deceased_indicator owns_car  ...  state    country  \
0      Mass Customer                  N      Yes  ...    QLD  Australia   
1      Mass Customer                  N       No  ...    NSW  Australia   
2  Affluent Customer                  N       No  ...    VIC  Australia   
3  Affluent Customer                  N      Yes  ...    QLD  Australia   
4  Affluent Customer                  N       No  ...    NSW  Australia   

   property_valuation Unnamed: 16 Unnamed: 17  Unnamed: 18  Unnamed: 19  \
0                   6        1.05      1.3125     1.640625     1.394531   
1                  11        1.10      1.1000     1.375000     1.168750   
2                   5        0.61      0.6100     0.610000     0.610000   
3                   1        1.03      1.2875     1.287500     1.287500   
4                   9        0.93      0.9300     1.162500     1.162500   

   Unnamed: 20  Rank     Value  
0            1     1  1.718750  
1            1     1  1.718750  
2            1     1  1.718750  
3            4     4  1.703125  
4            4     4  1.703125  

[5 rows x 23 columns]

# 1. New Customer List

# Rank

In [43]:
df_newcustomerlist['Rank'].sort_values().head()
#Em có thể thấy số 2,3,5,7,9 are missing

0    1
1    1
2    1
3    4
4    4
Name: Rank, dtype: int64

In [44]:
#Có cách nào để tìm số missing from sequence từ 1-1000?
print('min rank: ', df_newcustomerlist['Rank'].min())
print('max rank: ', df_newcustomerlist['Rank'].max())

min rank:  1
max rank:  1000


# Value 

In [45]:
#Converting unit of value from million to numbers
df_newcustomerlist['Value'] = df_newcustomerlist['Value']*1000000
df_newcustomerlist['Value'].head(3)

0    1718750.0
1    1718750.0
2    1718750.0
Name: Value, dtype: float64

In [46]:
#Converting float to int
df_newcustomerlist['Value'] = df_newcustomerlist['Value'].astype('int64')
df_newcustomerlist['Value'].head()


0    1718750
1    1718750
2    1718750
3    1703125
4    1703125
Name: Value, dtype: int64

In [60]:
#Cách để biết Rank và Value đều tỉ lệ nghịch? Nếu Rank chạy từ 1-1000 thì có Value nào giảm đi?
df_newcustomerlist.sort_values(by='Rank')[['Rank','Value']].head(20)
df_newcustomerlist['desc_rank'] = df_newcustomerlist['Value'].rank(ascending=False, method = 'min')
df_newcustomerlist[(df_newcustomerlist['Rank'] != df_newcustomerlist['desc_rank'])]

Empty DataFrame
Columns: [first_name, last_name, gender, past_3_years_bike_related_purchases, DOB, job_title, job_industry_category, wealth_segment, deceased_indicator, owns_car, tenure, address, postcode, state, country, property_valuation, Unnamed: 16, Unnamed: 17, Unnamed: 18, Unnamed: 19, Unnamed: 20, Rank, Value, arcending_rank, desc_rank]
Index: []

[0 rows x 25 columns]

In [69]:
df_newcustomerlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
first_name                             1000 non-null object
last_name                              971 non-null object
gender                                 1000 non-null object
past_3_years_bike_related_purchases    1000 non-null int64
DOB                                    983 non-null datetime64[ns]
job_title                              894 non-null object
job_industry_category                  835 non-null object
wealth_segment                         1000 non-null object
deceased_indicator                     1000 non-null object
owns_car                               1000 non-null object
tenure                                 1000 non-null int64
address                                1000 non-null object
postcode                               1000 non-null int64
state                                  1000 non-null object
country                                1000 non-null 

In [70]:
df_newcustomerlist.describe()

past_3_years_bike_related_purchases       tenure     postcode  \
count                          1000.000000  1000.000000  1000.000000   
mean                             49.836000    11.388000  3019.227000   
std                              27.796686     5.037145   848.895767   
min                               0.000000     0.000000  2000.000000   
25%                              26.750000     7.000000  2209.000000   
50%                              51.000000    11.000000  2800.000000   
75%                              72.000000    15.000000  3845.500000   
max                              99.000000    22.000000  4879.000000   

       property_valuation  Unnamed: 16  Unnamed: 17  Unnamed: 18  Unnamed: 19  \
count         1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean             7.397000     0.755880     0.848322     0.953801     0.880426   
std              2.758804     0.201735     0.243983     0.291860     0.277361   
min              1.000000     0.400000     0.400000     0.400000     0.340000   
25%              6.000000     0.590000     0.662500     0.712500     0.671875   
50%              8.000000     0.750000     0.838750     0.937500     0.859375   
75%              9.000000     0.920000     1.037500     1.162500     1.075781   
max             12.000000     1.100000     1.375000     1.718750     1.718750   

       Unnamed: 20         Rank        Value  arcending_rank    desc_rank  
count  1000.000000  1000.000000  1000.000000     1000.000000  1000.000000  
mean    498.819000   498.819000     0.881714      500.500000   498.819000  
std     288.810997   288.810997     0.293525      288.815975   288.810997  
min       1.000000     1.000000     0.340000        1.000000     1.000000  
25%     250.000000   250.000000     0.649531      250.500000   250.000000  
50%     500.000000   500.000000     0.860000      500.500000   500.000000  
75%     750.250000   750.250000     1.075000      750.500000   750.250000  
max    1000.000000  1000.000000     1.718750      999.000000  1000.000000

In [72]:
df_newcustomerlist[df_newcustomerlist.wealth_segment == 'Affluent Customer'].groupby('property_valuation')[[ 'Unnamed: 16', 'Unnamed: 17','Unnamed: 18','Unnamed: 19', 'Unnamed: 20','Value']].agg('mean')

Unnamed: 16  Unnamed: 17  Unnamed: 18  Unnamed: 19  \
property_valuation                                                       
1                      0.743333     0.866944     0.887431     0.887431   
2                      0.777857     0.925893     0.965536     0.965536   
3                      0.802941     0.900588     0.935699     0.935699   
4                      0.565556     0.608333     0.638333     0.638333   
5                      0.754286     0.848036     0.889062     0.889062   
6                      0.730588     0.813235     0.862059     0.862059   
7                      0.764483     0.877414     0.896142     0.896142   
8                      0.740500     0.828937     0.859234     0.859234   
9                      0.724545     0.830511     1.038139     1.038139   
10                     0.783846     0.855865     1.069832     1.069832   
11                     0.711429     0.821607     1.027009     1.027009   
12                     0.680000     0.766563     0.958203     0.958203   

                    Unnamed: 20     Value  
property_valuation                         
1                    404.333333  1.035358  
2                    448.071429  0.920243  
3                    494.411765  0.867890  
4                    509.888889  0.855208  
5                    546.857143  0.850098  
6                    583.941176  0.786445  
7                    534.241379  0.838578  
8                    450.625000  0.933005  
9                    502.136364  0.887094  
10                   634.884615  0.743442  
11                   433.357143  0.945027  
12                   577.875000  0.792813

In [67]:
test1 = df_newcustomerlist.groupby(['wealth_segment','property_valuation'])['Rank','Value'].agg(['min', 'max'])
test1 = test1.reset_index()

# multi layer column thì dùng dạng tuple 
#  correlation


test1.sort_values(['wealth_segment', 'property_valuation'])
#Em có thể thấy wealth_segment gồm Affuent (khá giả), High Net Worth (thượng lưu), Mass Customer (số đông), và sự phân chia
#của Property_Valuation từ 1-12 không trùng với min và max của Rank và Value. Mình có thể rank lại cột Rank dựa trên Value? và
#đó sắp xếp lại property_valuation?

wealth_segment property_valuation Rank           Value          
                                          min   max       min       max
0   Affluent Customer                  1    4   979  0.416500  1.703125
1   Affluent Customer                  2   89   997  0.357000  1.312500
2   Affluent Customer                  3  146   939  0.467500  1.225000
3   Affluent Customer                  4  206   879  0.537500  1.137500
4   Affluent Customer                  5    1   913  0.499375  1.718750
5   Affluent Customer                  6  114   974  0.425000  1.275000
6   Affluent Customer                  7   68   996  0.374000  1.354688
7   Affluent Customer                  8   16   963  0.440000  1.562500
8   Affluent Customer                  9    4   967  0.435625  1.703125
9   Affluent Customer                 10   89   985  0.408000  1.312500
10  Affluent Customer                 11   19   725  0.669375  1.531250
11  Affluent Customer                 12  158   974  0.425000  1.187500
12     High Net Worth                  1  289   967  0.435625  1.037500
13     High Net Worth                  2  375   920  0.493000  0.960000
14     High Net Worth                  3  163   937  0.470000  1.182031
15     High Net Worth                  4   46   928  0.480000  1.407812
16     High Net Worth                  5   44   832  0.575000  1.421094
17     High Net Worth                  6   10   921  0.490000  1.640625
18     High Net Worth                  7    6   944  0.460000  1.671875
19     High Net Worth                  8   30   988  0.399500  1.460938
20     High Net Worth                  9   72   947  0.459000  1.350000
21     High Net Worth                 10   19   924  0.488750  1.531250
22     High Net Worth                 11   26   948  0.456875  1.468750
23     High Net Worth                 12   46   967  0.435625  1.407812
24      Mass Customer                  1  104   888  0.525000  1.287500
25      Mass Customer                  2   26   997  0.357000  1.468750
26      Mass Customer                  3  104   993  0.391000  1.287500
27      Mass Customer                  4   23   994  0.382500  1.500000
28      Mass Customer                  5    8   983  0.410000  1.656250
29      Mass Customer                  6    1   988  0.399500  1.718750
30      Mass Customer                  7    6   988  0.399500  1.671875
31      Mass Customer                  8   12   987  0.400000  1.625000
32      Mass Customer                  9   21  1000  0.340000  1.515625
33      Mass Customer                 10    8   997  0.357000  1.656250
34      Mass Customer                 11    1   983  0.410000  1.718750
35      Mass Customer                 12   50   994  0.382500  1.406250

In [73]:
#rank

#rank over partition
test1['wealth_segment_Rank'] = test1.groupby('wealth_segment')[[('Value', 'max')]].rank(ascending=False, method='first')
test1.sort_values(['wealth_segment', 'wealth_segment_Rank'])


wealth_segment property_valuation Rank           Value            \
                                          min   max       min       max   
4   Affluent Customer                  5    1   913  0.499375  1.718750   
0   Affluent Customer                  1    4   979  0.416500  1.703125   
8   Affluent Customer                  9    4   967  0.435625  1.703125   
7   Affluent Customer                  8   16   963  0.440000  1.562500   
10  Affluent Customer                 11   19   725  0.669375  1.531250   
6   Affluent Customer                  7   68   996  0.374000  1.354688   
1   Affluent Customer                  2   89   997  0.357000  1.312500   
9   Affluent Customer                 10   89   985  0.408000  1.312500   
5   Affluent Customer                  6  114   974  0.425000  1.275000   
2   Affluent Customer                  3  146   939  0.467500  1.225000   
11  Affluent Customer                 12  158   974  0.425000  1.187500   
3   Affluent Customer                  4  206   879  0.537500  1.137500   
18     High Net Worth                  7    6   944  0.460000  1.671875   
17     High Net Worth                  6   10   921  0.490000  1.640625   
21     High Net Worth                 10   19   924  0.488750  1.531250   
22     High Net Worth                 11   26   948  0.456875  1.468750   
19     High Net Worth                  8   30   988  0.399500  1.460938   
16     High Net Worth                  5   44   832  0.575000  1.421094   
15     High Net Worth                  4   46   928  0.480000  1.407812   
23     High Net Worth                 12   46   967  0.435625  1.407812   
20     High Net Worth                  9   72   947  0.459000  1.350000   
14     High Net Worth                  3  163   937  0.470000  1.182031   
12     High Net Worth                  1  289   967  0.435625  1.037500   
13     High Net Worth                  2  375   920  0.493000  0.960000   
29      Mass Customer                  6    1   988  0.399500  1.718750   
34      Mass Customer                 11    1   983  0.410000  1.718750   
30      Mass Customer                  7    6   988  0.399500  1.671875   
28      Mass Customer                  5    8   983  0.410000  1.656250   
33      Mass Customer                 10    8   997  0.357000  1.656250   
31      Mass Customer                  8   12   987  0.400000  1.625000   
32      Mass Customer                  9   21  1000  0.340000  1.515625   
27      Mass Customer                  4   23   994  0.382500  1.500000   
25      Mass Customer                  2   26   997  0.357000  1.468750   
35      Mass Customer                 12   50   994  0.382500  1.406250   
24      Mass Customer                  1  104   888  0.525000  1.287500   
26      Mass Customer                  3  104   993  0.391000  1.287500   

   wealth_segment_Rank  
                        
4                  1.0  
0                  2.0  
8                  3.0  
7                  4.0  
10                 5.0  
6                  6.0  
1                  7.0  
9                  8.0  
5                  9.0  
2                 10.0  
11                11.0  
3                 12.0  
18                 1.0  
17                 2.0  
21                 3.0  
22                 4.0  
19                 5.0  
16                 6.0  
15                 7.0  
23                 8.0  
20                 9.0  
14                10.0  
12                11.0  
13                12.0  
29                 1.0  
34                 2.0  
30                 3.0  
28                 4.0  
33                 5.0  
31                 6.0  
32                 7.0  
27                 8.0  
25                 9.0  
35                10.0  
24                11.0  
26                12.0

# Address

In [50]:
# Em dùng groupby sort state và tìm min max của postcode trong từng state
test1= df_newcustomerlist.groupby(['state'])['postcode'].agg(['min', 'max', 'count','sum'])
test1.head()
#https://en.wikipedia.org/wiki/Postcodes_in_Australia có chứa bảng của postcode trong các state ở nước úc
#Anh có thể chỉ em scrapping data cái bảng đó lên đây và sau đó so sánh nếu nó có trùng?

# beautifulsoup

min   max  count      sum
state                            
NSW    2000  2880    506  1160902
QLD    4000  4879    228   976988
VIC    3000  3981    266   881337

In [92]:
df_newcustomerlist['postcode'].head(10)

0    4500
1    2113
2    3505
3    4814
4    2093
5    4075
6    2620
7    4814
8    2200
9    4151
Name: postcode, dtype: int64

In [99]:
if df_newcustomerlist.postcode > 5000:
    df_newcustomerlist['postcode_compare'] = 'VND'
else:
    df_newcustomerlist['postcode_compare'] = 'NAV'

df_newcustomerlist.head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Wealth Segment

first_name  last_name  gender  past_3_years_bike_related_purchases  \
0    Chickie    Brister    Male                                   86   
1      Morly     Genery    Male                                   69   
2    Ardelis  Forrester  Female                                   10   
3     Lucine      Stutt  Female                                   64   
4    Melinda     Hadlee  Female                                   34   

         DOB                   job_title job_industry_category  \
0 1957-07-12             General Manager         Manufacturing   
1 1970-03-22         Structural Engineer              Property   
2 1974-08-28      Senior Cost Accountant    Financial Services   
3 1979-01-28  Account Representative III         Manufacturing   
4 1965-09-21           Financial Analyst    Financial Services   

      wealth_segment deceased_indicator owns_car  ...  property_valuation  \
0      Mass Customer                  N      Yes  ...                   6   
1      Mass Customer                  N       No  ...                  11   
2  Affluent Customer                  N       No  ...                   5   
3  Affluent Customer                  N      Yes  ...                   1   
4  Affluent Customer                  N       No  ...                   9   

  Unnamed: 16  Unnamed: 17 Unnamed: 18 Unnamed: 19  Unnamed: 20  Rank  \
0        1.05       1.3125    1.640625    1.394531            1     1   
1        1.10       1.1000    1.375000    1.168750            1     1   
2        0.61       0.6100    0.610000    0.610000            1     1   
3        1.03       1.2875    1.287500    1.287500            4     4   
4        0.93       0.9300    1.162500    1.162500            4     4   

      Value  arcending_rank  desc_rank  
0  1.718750           999.0        1.0  
1  1.718750           999.0        1.0  
2  1.718750           999.0        1.0  
3  1.703125           996.5        4.0  
4  1.703125           996.5        4.0  

[5 rows x 25 columns]

In [51]:
test2 = df_newcustomerlist.groupby(['wealth_segment','job_industry_category'])['tenure','Value'].agg(['min', 'max', 'mean', 'sum'])
test2 = test2.reset_index()
#test2.sort_values(by='sum',ascending=False)
test2.sort_values(('Value', 'sum'))
# Có cách nào để em sort sum của Value không ạ? Nó cứ báo em bị lỗi

wealth_segment job_industry_category tenure                       \
                                               min max       mean   sum   
9      High Net Worth           Argiculture      6  14  10.000000    20   
8   Affluent Customer    Telecommunications      1  17   8.000000    32   
0   Affluent Customer           Argiculture      5  16  11.600000    58   
1   Affluent Customer         Entertainment      5  18  13.428571    94   
13     High Net Worth                    IT      0  19   9.500000    76   
26      Mass Customer    Telecommunications      9  18  11.400000   114   
17     High Net Worth    Telecommunications      3  17  10.454545   115   
4   Affluent Customer                    IT      0  21  10.307692   134   
19      Mass Customer         Entertainment      3  22  12.400000   186   
15     High Net Worth              Property      4  19  10.615385   138   
6   Affluent Customer              Property      4  20  10.714286   150   
16     High Net Worth                Retail      3  20  11.846154   154   
10     High Net Worth         Entertainment      1  22  11.266667   169   
7   Affluent Customer                Retail      4  21  12.200000   244   
18      Mass Customer           Argiculture      4  20  12.473684   237   
22      Mass Customer                    IT      2  21  10.300000   309   
3   Affluent Customer                Health      4  22  12.763158   485   
12     High Net Worth                Health      2  21  11.707317   480   
24      Mass Customer              Property      3  22  11.027027   408   
11     High Net Worth    Financial Services      2  22  12.772727   562   
25      Mass Customer                Retail      1  20  10.422222   469   
2   Affluent Customer    Financial Services      1  22  11.365385   591   
5   Affluent Customer         Manufacturing      1  22  10.980392   560   
14     High Net Worth         Manufacturing      1  21  12.000000   732   
21      Mass Customer                Health      1  21  10.958904   800   
23      Mass Customer         Manufacturing      2  21  11.287356   982   
20      Mass Customer    Financial Services      2  22  11.046729  1182   

     Value                                    
       min      max           mean       sum  
9   730000  1062500  896250.000000   1792500  
8   440000  1275000  949531.250000   3798125  
0   525000  1175000  842843.600000   4214218  
1   625000  1325000  899696.142857   6297873  
13  557812  1362500  831445.250000   6651562  
26  478125  1609375  825268.600000   8252686  
17  492999  1168750  818840.727273   9007248  
4   408000  1155468  804576.846154  10459499  
19  391000  1094375  760351.866667  11405278  
15  561000  1468750  882689.615385  11474965  
6   435624  1362500  843437.357143  11808123  
16  460000  1640625  953677.615385  12397809  
10  480000  1671875  944266.466667  14163997  
7   637500  1262500  863435.750000  17268715  
18  416500  1406250  935299.157895  17770684  
22  400000  1640625  864138.500000  25924155  
3   467500  1337500  851853.526316  32370434  
12  435624  1460937  823408.390244  33759744  
24  340000  1718750  945690.810811  34990560  
11  470000  1460937  924813.704545  40691803  
25  382500  1656250  925731.800000  41657931  
2   357000  1718750  840724.557692  43717677  
5   478125  1703125  912929.431373  46559401  
14  399500  1531250  851738.590164  51956054  
21  357000  1593750  859054.205479  62710957  
23  399500  1718750  902779.988506  78541859  
20  357000  1671875  875425.392523  93670517

# 2. Transactions

In [52]:
df_transactions = pd.read_excel('KPMG.xlsx', sheet_name= 'Transactions')
#comparation of min and max between list_price and standard_cost
test = df_transactions.groupby(['product_line','product_class','product_size'])['list_price','standard_cost'].agg(['min','max','sum'])
test = test.reset_index()
test['compared'] = test[('standard_cost','sum')] / test[('list_price','sum')]
test
#Em muốn dùng hàm loc hay get_item hay hàm điều kiện đễ lấy ra những rows mà max/min/sum của 'list_price' gấp đôi 
#max/min/sum của 'standard_cost' ạ? 

product_line product_class product_size list_price                       \
                                                  min      max         sum   
0      Mountain           low       medium     574.64   574.64   128144.72   
1      Mountain           low        small     688.63   688.63   137726.00   
2          Road          high        large      12.01   774.53   150277.18   
3          Road           low       medium     980.37   980.37   197054.37   
4          Road           low        small     590.26  1720.70   590615.72   
5          Road        medium        large    1240.31  1894.19  1028840.61   
6          Road        medium       medium     290.62  1538.99  1397589.06   
7          Road        medium        small    1703.52  1810.00   677151.20   
8      Standard          high        large    1842.92  1842.92   350154.80   
9      Standard          high       medium     227.88  2005.66  2099212.89   
10     Standard          high        small    1661.92  1977.36   689716.56   
11     Standard           low       medium      71.16  1793.43  1450065.55   
12     Standard        medium        large     202.62  2091.47  2846312.62   
13     Standard        medium       medium      60.34  1992.93  6672426.52   
14     Standard        medium        small     175.89  1775.81  1527055.02   
15      Touring           low       medium    1073.07  1073.07   245733.03   
16      Touring        medium        large    1362.99  2083.94  1477870.05   
17      Touring        medium       medium    1466.68  1466.68   275735.84   

   standard_cost                         compared  
             min      max           sum            
0         459.71   459.71  1.025153e+05  0.799997  
1         612.88   612.88  1.225760e+05  0.889999  
2           7.21   464.72  9.016747e+04  0.600008  
3         234.43   234.43  4.712043e+04  0.239124  
4         525.33  1531.42  5.256466e+05  0.889998  
5         598.76   820.78  4.608469e+05  0.447928  
6         206.35   829.65  9.146857e+05  0.654474  
7        1516.13  1610.90  6.026641e+05  0.889999  
8        1105.75  1105.75  2.100925e+05  0.599999  
9         136.73  1203.40  1.259528e+06  0.600000  
10       1479.11  1759.85  6.138479e+05  0.890000  
11         56.93   748.90  5.331041e+05  0.367641  
12        151.96   993.66  1.079133e+06  0.379134  
13         13.44   954.82  2.445840e+06  0.366559  
14        131.92  1580.47  1.354449e+06  0.886968  
15        933.84   933.84  2.138494e+05  0.870251  
16         57.74   863.95  3.670407e+05  0.248358  
17        363.25   363.25  6.829100e+04  0.247668